In [1]:
import os, os.path
import pandas as pd
import numpy as np, os 
from tqdm import tqdm
from collections import Counter

from torch.utils.data import Dataset, DataLoader
import torch


In [2]:
list_txt = os.listdir("train") # dir is your directory path
number_files = len(list_txt)
number_files

15594

In [3]:
type(list_txt)

list

In [4]:
index_txt = list_txt.index('0A0AA9C21C5D.txt')
index_txt                   

6804

In [5]:
os.getcwd()

'/home/arno/Documents/1_Study/Project/NLP_sentiment_pred'

In [6]:
def open_text(text):
    with open('train/'+text+'.txt') as f:
        text = f.read()
    # print out the first 100 characters
    return text

# NLP Solution

In [7]:
train_csv = pd.read_csv('train.csv')

In [8]:
print(train_csv.shape )
train_csv.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [9]:
train_csv.iloc[0]

id                                                         423A1CA112E2
discourse_id                                            1622627660524.0
discourse_start                                                     8.0
discourse_end                                                     229.0
discourse_text        Modern humans today are always on their phone....
discourse_type                                                     Lead
discourse_type_num                                               Lead 1
predictionstring      1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
Name: 0, dtype: object

In [10]:
res_text = open_text(train_csv.iloc[0][0])
predic_string = train_csv.iloc[0][7]

In [92]:
# list of counters
lead_counts = Counter()
position_counts = Counter()
claim_counts = Counter()
counterclaim_counts = Counter()
rebuttal_counts = Counter()
evidence_counts = Counter()
conclustate_counts = Counter()

In [93]:
speech = res_text[8:229]
for ind_pred in predic_string.split(' '):
    ind_pred_int = int(ind_pred)
    speech_split = speech.split(' ')
    lead_counts[speech_split[ind_pred_int-1]] +=1


In [14]:
lead_counts

Counter({'Modern': 1,
         'humans': 1,
         'today': 1,
         'are': 2,
         'always': 2,
         'on': 3,
         'their': 2,
         'phone.': 1,
         'They': 2,
         'phone': 1,
         'more': 1,
         'than': 1,
         '5': 1,
         'hours': 1,
         'a': 1,
         'day': 1,
         'no': 1,
         'stop': 1,
         '.All': 1,
         'they': 1,
         'do': 2,
         'is': 1,
         'text': 1,
         'back': 1,
         'and': 2,
         'forward': 1,
         'just': 1,
         'have': 1,
         'group': 1,
         'Chats': 1,
         'social': 1,
         'media.': 1,
         'even': 1,
         'it': 1,
         'while': 1,
         'driving.': 1})

In [15]:
speech.split()[0]

'Modern'

In [16]:
test_names, test_texts = [], []
for f in os.listdir('test'):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts.head()

,id,text
0,D46BCB48440A,"When people ask for advice,they sometimes talk..."
1,18409261F5C2,80% of Americans believe seeking multiple opin...
2,D72CB1C11673,Making choices in life can be very difficult. ...
3,DF920E0A7337,Have you ever asked more than one person for h...
4,0FB0700DAF44,"During a group project, have you ever asked a ..."


# Train CSV review and creation of counters


In [107]:

train_csv[10:20]

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
10,A8445CABFECE,1.622576e+12,18.0,85.0,Drivers should not be able to use phones while...,Position,Position 1,3 4 5 6 7 8 9 10 11 12 13 14
11,A8445CABFECE,1.622576e+12,86.0,202.0,Drivers who used their phone while operating a...,Claim,Claim 1,15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 3...
12,A8445CABFECE,1.622576e+12,203.0,1030.0,According to an article by the Edgar Snyder Fi...,Evidence,Evidence 1,36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 5...
13,A8445CABFECE,1.622576e+12,1031.0,1243.0,"In conclusion, drivers should not able to work...",Concluding Statement,Concluding Statement 1,177 178 179 180 181 182 183 184 185 186 187 18...
14,6B4F7A0165B9,1.622644e+12,36.0,512.0,The ability to stay connected to people we kno...,Lead,Lead 1,5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
15,6B4F7A0165B9,1.622644e+12,513.0,593.0,"However, this device is taken to areas that i...",Position,Position 1,92 93 94 95 96 97 98 99 100 101 102 103 104
16,6B4F7A0165B9,1.622644e+12,594.0,726.0,Within a vehicle capable of traveling upwards ...,Claim,Claim 1,105 106 107 108 109 110 111 112 113 114 115 11...
17,6B4F7A0165B9,1.622644e+12,727.0,1244.0,The most common of these distractions is a cel...,Evidence,Evidence 1,125 126 127 128 129 130 131 132 133 134 135 13...
18,6B4F7A0165B9,1.622644e+12,1245.0,1439.0,Conversely people may believe that laws in pre...,Counterclaim,Counterclaim 1,221 222 223 224 225 226 227 228 229 230 231 23...
19,6B4F7A0165B9,1.622644e+12,1440.0,1681.0,The problem becomes apparent that this is a se...,Rebuttal,Rebuttal 1,254 255 256 257 258 259 260 261 262 263 264 26...


In [70]:
train_csv.discourse_type.describe(include='all')

count     144293
unique         7
top        Claim
freq       50208
Name: discourse_type, dtype: object

In [156]:
def creation_count(train_csv):
    # list of counters
    lead_counts = Counter()
    position_counts = Counter()
    claim_counts = Counter()
    counterclaim_counts = Counter()
    rebuttal_counts = Counter()
    evidence_counts = Counter()
    conclustate_counts = Counter()

    for i in range(len(train_csv.discourse_id)):
        text = train_csv.discourse_text[i]
        text = text.split(' ')
        if train_csv.discourse_type[i] == 'Lead':
            for words_ind in range(len(text)):
                lead_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Position':
            for words_ind in range(len(text)):
                position_counts[text[words_ind-1]] += 1
            
        elif train_csv.discourse_type[i] == 'Claim':
            for words_ind in range(len(text)):
                claim_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Counterclaim':
            for words_ind in range(len(text)):
                counterclaim_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Rebuttal':
            for words_ind in range(len(text)):
                rebuttal_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Evidence':
            for words_ind in range(len(text)):
                evidence_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Concluding Statement':
            for words_ind in range(len(text)):
                conclustate_counts[text[words_ind-1]] += 1
    
    return lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts


In [158]:
lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts = creation_count(train_csv)